#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [3]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


Распакуем наши данные из архива.

In [4]:
!unzip ...

'unzip' is not recognized as an internal or external command,
operable program or batch file.


Загрузим данные из csv файла в датафрейм.

In [57]:
import pandas as pd

df = pd.read_csv('input_data.csv', sep= ';')


df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


Отобразим на карте координаты наших построек.

In [58]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]

# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [59]:
import math

def haversine_distance(latitude_start, longitude_start, latitude_end, longitude_end):
    """
    Calculates the distance between two points on the Earth's surface 
    using the Haversine formula.
    """

    earth_radius = 6371.0 

    # Convert degrees to radians
    lat1_rad = math.radians(latitude_start)
    lon1_rad = math.radians(longitude_start)
    lat2_rad = math.radians(latitude_end)
    lon2_rad = math.radians(longitude_end)

    # Difference in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate distance
    distance = earth_radius * c

    return distance

moscow_center = (55.7558, 37.6173)
spb_center = (59.9343, 30.3351)

df['is_Moscow'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1]) <= 20, axis=1)
df['is_Saint_Petersburg'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], spb_center[0], spb_center[1]) <= 20, axis=1)


df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,False,False


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [60]:
df = df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1)

df.head()


,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_Saint_Petersburg

Числовые: price, level, levels, rooms, area, kitchen_area, geo_lat, geo_lon, building_type, object_type, postal_code, street_id, id_region, house_id

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [61]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

# Преобразуем булевы значения в числа (0 и 1)
df['is_Moscow'] = df['is_Moscow'].astype(int)
df['is_Saint_Petersburg'] = df['is_Saint_Petersburg'].astype(int)

# cols - столбцы, которые нужно преобразовать
ohe_encoder = ce.OneHotEncoder(cols = ['building_type'])
df = ohe_encoder.fit_transform(df)

df.head()


,date,price,level,levels,rooms,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,building_type_7,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,1,0,0,0,0,0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,1,0,0,0,0,0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,0,1,0,0,0,0,0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,1,0,0,0,0,0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,0,1,0,0,0,0,0,0,0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [62]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df['date'] = pd.to_datetime(df['date']) 

earliest_date = df['date'].min() 
df['days_passed'] = (df['date'] - earliest_date).dt.days 

df['levels'] = df['levels'].replace(0, 1)

# Вычисляем коэффициент этажа относительно общего числа этажей 
df['level_proportion'] = df['level'] / df['levels'] 

 
scaler_instance = StandardScaler() 
features_to_scale = ['days_passed', 'level_proportion', 'price', 'level', 'levels'] 
df[features_to_scale] = scaler_instance.fit_transform(df[features_to_scale]) 

df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,building_type_7,is_Moscow,is_Saint_Petersburg,days_passed,level_proportion
0,2021-01-01,-0.021932,1.622769,2.665110,1,30.3,0.0,1,0,0,0,0,0,0,0,0,-1.858355,-0.288681
1,2021-01-01,-0.026996,-0.270043,-0.936923,1,33.0,6.0,1,0,0,0,0,0,0,0,0,-1.858355,1.337255
2,2021-01-01,0.019789,-0.459324,0.171395,3,85.0,12.0,0,1,0,0,0,0,0,0,0,-1.858355,-0.843688
3,2021-01-01,-0.018651,-0.648605,-0.936923,3,82.0,9.0,1,0,0,0,0,0,0,0,0,-1.858355,0.077155
4,2021-01-01,-0.021686,-0.837886,-1.214003,1,30.0,9.0,0,1,0,0,0,0,0,0,0,-1.858355,0.287172


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [73]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

class KNNRegressor:
  def __init__(self, k=5, distance_measure='euclidean'):

    self.k = k
    self.distance_measure = distance_measure
    self.training_data = None
    self.target_values = None

  def fit(self, X, y):

    #X (numpy.ndarray): Матрица признаков.
    #y (numpy.ndarray): Вектор целевых значений.

    self.training_data = np.array(X)
    self.target_values = np.array(y)

  def predict(self, X):

    # X (numpy.ndarray): Матрица признаков.
    # numpy.ndarray: Вектор предсказанных значений.

    X = np.array(X)
    distances = pairwise_distances(X, self.training_data, metric=self.distance_measure)
    nearest_neighbors = np.argsort(distances, axis=1)[:, :self.k]
    predictions = np.array([np.mean(self.target_values[neighbor_indices]) for neighbor_indices in nearest_neighbors])
    return predictions


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [76]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None  # Для Momentum
        self.accumulated_squared_grad = None  # Для AdaGrad

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Инициализация весов и смещения
        self.weights = np.zeros(n_features)
        self.bias = 0
        if self.optimization == 'Momentum':
            self.velocity = np.zeros(n_features)
        elif self.optimization == 'AdaGrad':
            self.accumulated_squared_grad = np.zeros(n_features)

        for iter in range(self.max_iter):
            for i in range(n_samples):
                # Прогноз
                y_predicted = np.dot(X[i], self.weights) + self.bias
                # Вычисление градиентов
                error = y_predicted - y[i]
                dw = (2 * error * X[i])  # Градиент по весам
                db = 2 * error  # Градиент по смещению

                # Обновление весов и смещения в зависимости от метода оптимизации
                if self.optimization == 'SGD':
                    self.weights -= self.learning_rate * dw
                    self.bias -= self.learning_rate * db
                elif self.optimization == 'Momentum':
                    self.velocity = self.decay_rate * self.velocity + (1 - self.decay_rate) * dw
                    self.weights -= self.learning_rate * self.velocity
                    self.bias -= self.learning_rate * db
                elif self.optimization == 'AdaGrad':
                    self.accumulated_squared_grad += dw**2
                    adjusted_lr = self.learning_rate / (np.sqrt(self.accumulated_squared_grad) + self.epsilon)
                    self.weights -= adjusted_lr * dw
                    self.bias -= adjusted_lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.








!!!
код написанный ниже не работает из-за слишком больших объёмов датасета, но я не знаю, как его оптимизировать :(

In [77]:
# Выбираем признаки для обучения модели (days_passed, level_proportion)
X = df[['days_passed', 'level_proportion']]

# Целевая переменная (цена)
y = df['price']

# Обучение модели
knn_regressor = KNNRegressor(k=5) # Выберите подходящее количество соседей
knn_regressor.fit(X, y)

# Оценка модели (RMSE)
rmse = mean_squared_error(y, knn_regressor.predict(X), squared=False)
print(f"RMSE: {rmse}")

# Сравниваем предсказанные значения с реальными
predictions = knn_regressor.predict(X)

# Создаем новый DataFrame для сравнения
comparison_df = pd.DataFrame({'Real_Price': y, 'Predicted_Price': predictions})

print(comparison_df.head(10))

MemoryError: Unable to allocate 939. TiB for an array with shape (11358150, 11358150) and data type float64